#### Import libraries  

In [11]:
import pandas as pd
import pyfpgrowth  # https://fp-growth.readthedocs.io/en/latest/

#### Parameters  

In [12]:
csv_in = 'groceries-col.csv'

#### Read CSV file  

In [15]:
df = pd.read_csv(csv_in, delimiter=',')
print(df.shape)
print(df.info())
display(df.head())

(9834, 32)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9834 entries, 0 to 9833
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   citrus fruit         9834 non-null   object
 1   semi-finished bread  7675 non-null   object
 2   margarine            6032 non-null   object
 3   ready soups          4733 non-null   object
 4   Unnamed: 4           3729 non-null   object
 5   Unnamed: 5           2874 non-null   object
 6   Unnamed: 6           2229 non-null   object
 7   Unnamed: 7           1684 non-null   object
 8   Unnamed: 8           1246 non-null   object
 9   Unnamed: 9           896 non-null    object
 10  Unnamed: 10          650 non-null    object
 11  Unnamed: 11          468 non-null    object
 12  Unnamed: 12          351 non-null    object
 13  Unnamed: 13          273 non-null    object
 14  Unnamed: 14          196 non-null    object
 15  Unnamed: 15          141 non-null    object


,citrus fruit,semi-finished bread,margarine,ready soups,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,whole milk,butter,yogurt,rice,abrasive cleaner,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_list= sorted(list(set(df.values.flatten())))

TypeError: '<' not supported between instances of 'str' and 'float'

#### Check appearance of each StockCode  

In [ ]:
top_sc = df['StockCode'].value_counts()
print(top_sc.size)
print(top_sc.head())

#### Format transaction data  
Before:  
```
invoice1, stockcode1, ...  
invoice1, stockcode2, ...  
invoice2, stockcode1, ...  
invoice3, stockcode1, ...  
invoice3, stockcode3, ...  
```

After:  
```
[  
  [stockcode1, stockcode2],    
  [stockcode1],  
  [stockcode1, stockcode3],  
  ...  
]  
```

and stockcode should be interger.  

#### Assign an integer (ID) to each stockcode  

In [ ]:
id2sc = sorted(list(set(df['StockCode'])))
sc2id = {}
for i in range(len(id2sc)):
    sc2id[id2sc[i]] = i

In [ ]:
df['StockCode_ID'] = df['StockCode'].map(lambda x: sc2id[x])
display(df.head())

#### Grouping transaction data by InvoiceNo  

In [ ]:
invoices = []
for r in df.groupby('InvoiceNo'):
    #print(len(r))  # debug
    #print(r[0])  # debug
    #print(r[1])  # debug
    #break  # debug
    s1 = set(r[1]['StockCode_ID'])
    invoices.append(list(s1))
print(len(invoices))

#### Market Basket Analysis by FP-Growth  

In [ ]:
%time patterns = pyfpgrowth.find_frequent_patterns(invoices, 40)

In [ ]:
#print(patterns)

In [ ]:
%time rules = pyfpgrowth.generate_association_rules(patterns, 0.8)

In [ ]:
print(rules)

In [ ]:
results = []
for x in rules:
    ret = [x, rules[x][0], rules[x][1]]
    results.append(ret)
df_res = pd.DataFrame(results)
df_res.columns = ['LHS', 'RHS', 'Conf']

In [ ]:
display(df_res.sort_values(by='Conf', ascending=False))

#### Get original StockCode  

In [ ]:
print(id2sc[1474])
print(id2sc[1475])
print(id2sc[1478])

#### Calculation of Lift  

In [ ]:
n_all = len(invoices)
lift = []
for i in range(df_res.shape[0]):
    rhs = df_res.at[i, 'RHS']
    conf = df_res.at[i, 'Conf']
    n_rhs = 0
    for items in invoices:
        if set(items) >= set(rhs):
            n_rhs += 1
    lift1 = conf / (n_rhs / n_all)
    lift.append(lift1)
    
df_res['Lift'] = lift

In [ ]:
display(df_res.sort_values(by='Conf', ascending=False))